In [120]:
import urllib.request
import json 
from bs4 import BeautifulSoup
#from datetime import date
from dateutil import parser


ModuleNotFoundError: No module named 'bson'

In [83]:
#https://www.cbc.ca/search_api/v1/search?q=mortgage%20rate&sortOrder=relevance&page=100&fields=feed


def get_initial_url(search_term):
    """returns the URL of the first page API call given a search string"""
    
    words = search_term.split()
    url_prefix = "https://www.cbc.ca/search_api/v1/search?"
    query = "q=" + "%20".join(words)
    url_suffix = "&sortOrder=relevance&page=1&fields=feed"
    
    first_url = url_prefix + query + url_suffix
    
    print("FIRST URL API CALL: ", first_url)
    
    return first_url
    
    
get_initial_url("interest rate index")

FIRST URL API CALL:  https://www.cbc.ca/search_api/v1/search?q=interest%20rate%20index&sortOrder=relevance&page=1&fields=feed


'https://www.cbc.ca/search_api/v1/search?q=interest%20rate%20index&sortOrder=relevance&page=1&fields=feed'

In [84]:
first_url = get_initial_url("interest rate")
with urllib.request.urlopen(first_url) as url:
    data = json.loads(url.read().decode())
    for article_dict in data:
        print(article_dict["url"])
        article_url = url
        print("----")
        #break

FIRST URL API CALL:  https://www.cbc.ca/search_api/v1/search?q=interest%20rate&sortOrder=relevance&page=1&fields=feed
//www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768
----
//www.cbc.ca/player/play/1707273795558
----
//www.cbc.ca/player/play/1706560067565
----
//www.cbc.ca/news/canada/british-columbia/programs/onthecoast/march-5-2020-interest-rates-1.5487903
----
//www.cbc.ca/player/play/1707557955587
----
//www.cbc.ca/news/canada/ottawa/programs/allinaday/money-column-interest-rate-cut-1.5486004
----
//www.cbc.ca/player/play/1706421827523
----
//www.cbc.ca/news/business/interest-rates-coronavirus-evans-1.5515884
----
//www.cbc.ca/news2/interactives/interest-rates/
----
//www.cbc.ca/player/play/1706366019839
----


In [131]:

def extract_json_items(url, specify_source_type=True):
    """Returns a json containing the following items from a CBC article:
        url: the url of the article
        urlToImage: the url of the header image
        title: the title of the article 
        description: subheader of the article
        author: author (note that some articles do not specify author)
        source: CBC if specify_source_type == False, subdivision of CBC if True (ex: "CBC radio")
        publishedAt: tuple of (date_string, datetime object)
        
        input: url returned from CBC API in "url" field (missing "http:" as part of URL)
    """
    json_dict = {}
    article_url = "http:" + url
    
    #get HTML from article URL into BeautifulSoup
    html_bytes = urllib.request.urlopen(article_url)
    mybytes = html_bytes.read()
    html = mybytes.decode("utf8")
    html_bytes.close()
    soup = BeautifulSoup(html, 'html.parser')

    #TITLE in detailHeadline (h1 tag)
    title_h1 = soup.find("h1", {"class": "detailHeadline"})
    #print("TITLE: ", title_h1.text)
    json_dict["title"] = title_h1.text

    #DESCRIPTION IN deck (h2 tag)
    desc_h2 = soup.find("h2", {"class": "deck"})
    #print("DESCRIPTION: ", desc_h2.text)
    json_dict["description"] = desc_h2.text


    #AUTHOR in authorText span
    author_span = soup.find("span", {"class": "authorText"})
    print(author_span)
    if author_span:
        json_dict["author"] = author_span.text
    else:
        json_dict["author"] = None


    #source appears before <span class="bullet"> · </span>
    #if author is attached, there are two bullet tags
    #if no author attached, there is one bullet tag
    if specify_source_type:
        bullet_spans = soup.find_all("span", {"class": "bullet"})
        for bullet_span in bullet_spans:
            previous_str = str(bullet_span.previous_sibling)
            if previous_str.startswith("CBC"):
                source = previous_str
    else:
        source = "CBC"
    json_dict["source"] = source

    time_tag = soup.find("time", {"class": "timeStamp"})
    datetime_str = time_tag.attrs["datetime"]
    datetime_obj = parser.isoparse(datetime_str)

    time_str = time_tag.text.split("|")[0].replace("Posted: ", "").strip()
    
    #ERROR: if we want to return a datetime object, error when writing to JSON
    #json_dict["publishedAt"] = (time_str, datetime_obj)
    #write as string for now
    json_dict["publishedAt"] = (time_str, datetime_str)


    main_image_tag = soup.find("figure", {"class": "imageMedia leadmedia-story full"})
    main_image_url = main_image_tag.find("img").attrs["src"]
    json_dict["urlToImage"] = main_image_url
    
    json_dict["url"] = article_url

    #print("ARTICLE URL: ", article_url)
#     for key, item in json_dict.items():
#         print(key + ":")
#         print(item)
#         print("-----")
    
    final_json = json.dumps(json_dict)
    return final_json


    
    

In [136]:
test_url = "//www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768"
radio_url = "//www.cbc.ca/radio/costofliving/slashed-interest-rates-getting-a-piece-of-the-electric-car-pie-and-a-happy-jobs-friday-to-all-1.5486253"
media_url = "//www.cbc.ca/player/play/1707317315674"
noauthor_url = "//www.cbc.ca/news/canada/coronavirus-covid19-world-canada-may12-1.5564261"

extract_json_items(test_url)

<span class="authorText"><a class="" href="/news/politics/john-paul-tasker-1.3389288">John Paul Tasker</a></span>


'{"title": "Federal deficit likely to be higher than $252 billion, parliamentary budget\\u00a0officer says", "description": "PBO says it\'s possible federal debt will hit $1 trillion because of pandemic relief spending", "author": "John Paul Tasker", "source": "CBC News", "publishedAt": ["May 12, 2020 5:34 PM ET", "2020-05-12T22:23:31.746Z"], "urlToImage": "https://i.cbc.ca/1.5138746.1558027091!/fileImage/httpImage/image.jpg_gen/derivatives/16x9_780/phoenix-pay-system.jpg", "url": "http://www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768"}'